source: https://www.kaggle.com/ejunichi/m5-simple-fe from https://www.kaggle.com/ejunichi/m5-three-shades-of-dark-darker-magic

In [1]:
import sys
import os
import pathlib
import gc
import pandas as pd
pd.set_option('display.max_columns', 500)
# pd.set_option('display.max_rows', 500)
import numpy as np
import math
import random
import pickle
import time
import psutil
import warnings

# custom import
from sklearn.preprocessing import LabelEncoder
from multiprocessing import Pool        # Multiprocess Runs
from sklearn.preprocessing import PowerTransformer

# warnings.filterwarnings('ignore')

# constant variables for helper functions

In [2]:
N_CORES = psutil.cpu_count()     # Available CPU cores
print(f"N_CORES: {N_CORES}")

N_CORES: 16


# function nicely diplaying a head of Pandas DataFrame

In [3]:
import IPython

def display(*dfs, head=True):
    for df in dfs:
        IPython.display.display(df.head() if head else df)

# function fixing random seeds

In [4]:
def seed_everything(seed=0):
    """Sets seed to make all processes deterministic     # type: int
    
    """
    random.seed(seed)
    np.random.seed(seed)

SEED = 42
seed_everything(SEED)    

# function processing df in multiprocess

In [5]:
def run_df_in_multiprocess(func, t_split):
    """Process ds in Multiprocess
    
    """
    num_cores = np.min([N_CORES,len(t_split)])
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, t_split), axis=1)
    pool.close()
    pool.join()
    return df

# other helper functions

In [6]:
def get_memory_usage():
    """メモリ使用量を確認するためのシンプルな「メモリプロファイラ」
    
    """
    return np.round(psutil.Process(os.getpid()).memory_info()[0]/2.**30, 2) 
        
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)


def merge_by_concat(df1, df2, merge_on):
    """
    dtypesを失わないための連結による結合
    
    """
    
    merged_gf = df1[merge_on]
    merged_gf = merged_gf.merge(df2, on=merge_on, how='left')
    new_columns = [col for col in list(merged_gf) if col not in merge_on]
    df1 = pd.concat([df1, merged_gf[new_columns]], axis=1)
    return df1

#  constant variables for data import

In [7]:
# _DATA_DIR = os.path.sep.join(["data", "M5_Three_shades_of_Dark_Darker_magic", "sample"])
# _DATA_DIR = os.path.sep.join(["data", "M5_Three_shades_of_Dark_Darker_magic"])
_DATA_DIR = os.path.sep.join(["data", "M5_Three_shades_of_Dark_Darker_magic"])

_CALENDAR_CSV_FILE = "calendar.csv"
_SAMPLE_SUBMISSION_CSV_FILE = "sample_submission.csv"
# _SALES_TRAIN_VALIDATION_CSV_FILE = "sales_train_validation.csv"
_SALES_TRAIN_EVALUATION_CSV_FILE = "sales_train_evaluation.csv"
_SELL_PRICES_CSV_FILE = "sell_prices.csv"

# S3より取得済み。
_IS_RUN_ON_SAGEMAKER = False

# downlaod data (only on sagemaker)

In [8]:

if _IS_RUN_ON_SAGEMAKER:
    import sagemaker

    # print(sagemaker.s3.parse_s3_url('s3://sagemaker-m5-forecasting-okada/accuracy/original/calendar.csv'))
    parent_dir = pathlib.Path(os.path.abspath(os.curdir)).parent.parent
    local_path = os.path.sep.join([str(parent_dir), _DATA_DIR])
    print(local_path)

    def import_one_object_from_s3(s3_uri, local_path):

        sagemaker.s3.S3Downloader.download(
            s3_uri=s3_uri,
            local_path=local_path
        )
        !ls $local_path

    calendar_data_s3_uri = 's3://sagemaker-m5-forecasting-okada/accuracy/original/calendar.csv'
    sales_train_evaluation_data_s3_uri = 's3://sagemaker-m5-forecasting-okada/accuracy/original/sales_train_evaluation.csv'
    sales_train_validation_data_s3_uri = 's3://sagemaker-m5-forecasting-okada/accuracy/original/sales_train_validation.csv'
    sample_submission_data_s3_uri = 's3://sagemaker-m5-forecasting-okada/accuracy/original/sample_submission.csv'
    sell_prices_data_s3_uri = 's3://sagemaker-m5-forecasting-okada/accuracy/original/sell_prices.csv'

    s3_uris = [
        calendar_data_s3_uri,
        sales_train_evaluation_data_s3_uri,
        sales_train_validation_data_s3_uri,
        sample_submission_data_s3_uri,
        sell_prices_data_s3_uri
    ]

    for s3_uri in s3_uris:
        import_one_object_from_s3(s3_uri, local_path)


# import data

In [9]:
def reduce_mem_usage(df, verbose=True):
    """
    reduce the memory usage of the given dataframe.
    https://qiita.com/hiroyuki_kageyama/items/02865616811022f79754
    
    Args:
        df: Dataframe
        verbose: 
        
    Returns:
        df, whose memory usage is reduced.

    Raises:
        None
    """
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns: #columns毎に処理
        col_type = df[col].dtypes
        if col_type in numerics: #numericsのデータ型の範囲内のときに処理を実行. データの最大最小値を元にデータ型を効率的なものに変更
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

def read_csv_data(directory, file_name):
    print('Reading files...')
    df = pd.read_csv(os.path.sep.join([str(directory), _DATA_DIR, file_name]))
    df = reduce_mem_usage(df)
    print('{} has {} rows and {} columns'.format(file_name, df.shape[0], df.shape[1]))
    
    return df

# read csv data

In [10]:
parent_dir = pathlib.Path(os.path.abspath(os.curdir)).parent.parent
print(f"parent_dir: {parent_dir}")

df_sales_train_evaluation = read_csv_data(parent_dir, _SALES_TRAIN_EVALUATION_CSV_FILE)

parent_dir: /home/ec2-user/SageMaker
Reading files...
Mem. usage decreased to 96.13 Mb (78.8% reduction)
sales_train_evaluation.csv has 30490 rows and 1947 columns


In [11]:
df_sell_prices = read_csv_data(parent_dir, _SELL_PRICES_CSV_FILE)

Reading files...
Mem. usage decreased to 130.48 Mb (37.5% reduction)
sell_prices.csv has 6841121 rows and 4 columns


In [12]:
df_calendar = read_csv_data(parent_dir, _CALENDAR_CSV_FILE)

Reading files...
Mem. usage decreased to  0.12 Mb (41.9% reduction)
calendar.csv has 1969 rows and 14 columns


In [13]:
df_sample_submission = read_csv_data(parent_dir, _SAMPLE_SUBMISSION_CSV_FILE)

Reading files...
Mem. usage decreased to  2.09 Mb (84.5% reduction)
sample_submission.csv has 60980 rows and 29 columns


In [14]:
print(f"df_sales_train_evaluation: {df_sales_train_evaluation.head()}")
print(f"df_sample_submission: {df_sample_submission.head()}")

df_sales_train_evaluation:                               id        item_id    dept_id   cat_id store_id  \
0  HOBBIES_1_001_CA_1_evaluation  HOBBIES_1_001  HOBBIES_1  HOBBIES     CA_1   
1  HOBBIES_1_002_CA_1_evaluation  HOBBIES_1_002  HOBBIES_1  HOBBIES     CA_1   
2  HOBBIES_1_003_CA_1_evaluation  HOBBIES_1_003  HOBBIES_1  HOBBIES     CA_1   
3  HOBBIES_1_004_CA_1_evaluation  HOBBIES_1_004  HOBBIES_1  HOBBIES     CA_1   
4  HOBBIES_1_005_CA_1_evaluation  HOBBIES_1_005  HOBBIES_1  HOBBIES     CA_1   

  state_id  d_1  d_2  d_3  d_4  d_5  d_6  d_7  d_8  d_9  d_10  d_11  d_12  \
0       CA    0    0    0    0    0    0    0    0    0     0     0     0   
1       CA    0    0    0    0    0    0    0    0    0     0     0     0   
2       CA    0    0    0    0    0    0    0    0    0     0     0     0   
3       CA    0    0    0    0    0    0    0    0    0     0     0     0   
4       CA    0    0    0    0    0    0    0    0    0     0     0     0   

   d_13  d_14  d_15  d_16  d_

# constant variables for preprocessing/prediction

In [15]:


# 予測期間とitem数の定義 / number of items, and number of prediction period
_NUM_UNIQUE_ITEM_ID = df_sales_train_evaluation.shape[0]  # 30490
print(f"_NUM_UNIQUE_ITEM_ID: {_NUM_UNIQUE_ITEM_ID}")
_DAYS_FOR_PREDICTION = df_sample_submission.shape[1] - 1  # 28
print(f"_DAYS_FOR_PREDICTION: {_DAYS_FOR_PREDICTION}")

# DAYS_PER_YEAR = 365
# _NUM_YEARS_FOR_MELT = 2
# _NUM_IMPORT_ROWS_FOR_MELT = DAYS_PER_YEAR * _NUM_YEARS_FOR_MELT * _NUM_UNIQUE_ITEM_ID
# print(f"_NUM_IMPORT_ROWS_FOR_MELT: {_NUM_IMPORT_ROWS_FOR_MELT}")

_SALES_HISTORY_DAYS = 1913
_SALES_HISTORY_START_DAYS_FOR_VALIDATION = _SALES_HISTORY_DAYS + 1
_SALES_HISTORY_START_DAYS_FOR_EVALUATION = _SALES_HISTORY_START_DAYS_FOR_VALIDATION + _DAYS_FOR_PREDICTION
print(f"_SALES_HISTORY_START_DAYS_FOR_EVALUATION: {_SALES_HISTORY_START_DAYS_FOR_EVALUATION}")

TARGET = 'sales'
MAIN_INDEX = ['id','d']  # We can identify items by these columns

_NUM_UNIQUE_ITEM_ID: 30490
_DAYS_FOR_PREDICTION: 28
_SALES_HISTORY_START_DAYS_FOR_EVALUATION: 1942


# Create Grid from df_sales_train_evaluation by melting

In [16]:
# We can tranform horizontal representation to vertical "view"
# Our "index" will be 'id','item_id','dept_id','cat_id','store_id','state_id' and labels are 'd_' coulmns
index_columns = ['id','item_id','dept_id','cat_id','store_id','state_id']

grid_df = pd.melt(df_sales_train_evaluation, 
                  id_vars = index_columns, 
                  var_name = 'd', 
                  value_name = TARGET)

print(f"Train shapes: {df_sales_train_evaluation.shape}, {grid_df.shape}")

print(f"grid_df: {grid_df}")

Train shapes: (30490, 1947), (59181090, 8)
grid_df:                                      id        item_id    dept_id   cat_id  \
0         HOBBIES_1_001_CA_1_evaluation  HOBBIES_1_001  HOBBIES_1  HOBBIES   
1         HOBBIES_1_002_CA_1_evaluation  HOBBIES_1_002  HOBBIES_1  HOBBIES   
2         HOBBIES_1_003_CA_1_evaluation  HOBBIES_1_003  HOBBIES_1  HOBBIES   
3         HOBBIES_1_004_CA_1_evaluation  HOBBIES_1_004  HOBBIES_1  HOBBIES   
4         HOBBIES_1_005_CA_1_evaluation  HOBBIES_1_005  HOBBIES_1  HOBBIES   
...                                 ...            ...        ...      ...   
59181085    FOODS_3_823_WI_3_evaluation    FOODS_3_823    FOODS_3    FOODS   
59181086    FOODS_3_824_WI_3_evaluation    FOODS_3_824    FOODS_3    FOODS   
59181087    FOODS_3_825_WI_3_evaluation    FOODS_3_825    FOODS_3    FOODS   
59181088    FOODS_3_826_WI_3_evaluation    FOODS_3_826    FOODS_3    FOODS   
59181089    FOODS_3_827_WI_3_evaluation    FOODS_3_827    FOODS_3    FOODS   

         st

# テスト結果を格納するためのRowを追加

In [17]:
# add the test result receiver column to grid_df
df_test_result_receiver = pd.DataFrame()
for i in range(0,_DAYS_FOR_PREDICTION):
    temp_df = df_sales_train_evaluation[index_columns]
    temp_df = temp_df.drop_duplicates() # snince "id" is unique, this is not necessary
    temp_df['d'] = 'd_'+ str(_SALES_HISTORY_START_DAYS_FOR_EVALUATION + i)
    temp_df[TARGET] = np.nan
    df_test_result_receiver = pd.concat([df_test_result_receiver,temp_df])

print(f"df_test_result_receiver: {df_test_result_receiver}")

grid_df = pd.concat([grid_df, df_test_result_receiver])
grid_df = grid_df.reset_index(drop=True)

print(f"grid_df concatenated with df_test_result_receiver: {grid_df}")

# 一時的なDFを削除する
# We will not need original train_df anymore and can remove it
del temp_df, df_test_result_receiver
# del temp_df, df_test_result_receiver, df_sales_train_evaluation
gc.collect()

# check memory usage
print("{:>20}: {:>8}".format('Original grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

# change the data type from string into category for memory reduction
for col in index_columns:
    print(f"grid_df[col]: {grid_df[col]}")
    grid_df[col] = grid_df[col].astype('category')

# check memory usage
print("{:>20}: {:>8}".format('Reduced grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

df_test_result_receiver:                                   id        item_id    dept_id   cat_id  \
0      HOBBIES_1_001_CA_1_evaluation  HOBBIES_1_001  HOBBIES_1  HOBBIES   
1      HOBBIES_1_002_CA_1_evaluation  HOBBIES_1_002  HOBBIES_1  HOBBIES   
2      HOBBIES_1_003_CA_1_evaluation  HOBBIES_1_003  HOBBIES_1  HOBBIES   
3      HOBBIES_1_004_CA_1_evaluation  HOBBIES_1_004  HOBBIES_1  HOBBIES   
4      HOBBIES_1_005_CA_1_evaluation  HOBBIES_1_005  HOBBIES_1  HOBBIES   
...                              ...            ...        ...      ...   
30485    FOODS_3_823_WI_3_evaluation    FOODS_3_823    FOODS_3    FOODS   
30486    FOODS_3_824_WI_3_evaluation    FOODS_3_824    FOODS_3    FOODS   
30487    FOODS_3_825_WI_3_evaluation    FOODS_3_825    FOODS_3    FOODS   
30488    FOODS_3_826_WI_3_evaluation    FOODS_3_826    FOODS_3    FOODS   
30489    FOODS_3_827_WI_3_evaluation    FOODS_3_827    FOODS_3    FOODS   

      store_id state_id       d  sales  
0         CA_1       CA  d_1942  

# remove some 0 sale price rows which actually means that products do not exist yet

In [18]:
# 各train_dfアイテム行の先行ゼロ値は実際の0売上ではなく、店にアイテムがないことを意味する。
# そのようなゼロを削除することで、一部のメモリを安全にする。

# 価格は州ごとに設定されるので、 リリース週があまり正確ではありません
# find the oldest release week (= smallest wm_yr_wk) of each item at each store
item_release_week_df = df_sell_prices.groupby(['store_id','item_id'])['wm_yr_wk'].agg(['min']).reset_index()
# display(df_sell_prices.groupby(['store_id','item_id'])['wm_yr_wk'].agg(['min']))
# display(df_sell_prices.groupby(['store_id','item_id']).get_group(("CA_1", "FOODS_1_001")))
# display(df_sell_prices.groupby(['store_id','item_id']).get_group(("CA_1", "FOODS_1_001")).agg(['min']))
# display(df_sell_prices.groupby(['store_id','item_id'])['wm_yr_wk'].get_group(("CA_1", "FOODS_1_001")))

# just change the column name from "min" to "release"
item_release_week_df.columns = ['store_id','item_id','release']
print(f"item_release_week_df: {item_release_week_df}")

# concat with grid_d
grid_df = merge_by_concat(grid_df, item_release_week_df, ['store_id','item_id'])
del item_release_week_df
gc.collect()
print(f"grid_df after concatenating with item_release_week_df: {grid_df}")

# grid_dfから「ゼロ」行をいくつか削除したい  
# それを行うには、wm_yr_wk列が必要
# 部分的にcalendar_dfを結合
grid_df = merge_by_concat(grid_df, df_calendar[['wm_yr_wk','d']], ['d'])
print(f"grid_df after concatenating with df_calendar: {grid_df}")


item_release_week_df:       store_id          item_id  release
0         CA_1      FOODS_1_001    11101
1         CA_1      FOODS_1_002    11101
2         CA_1      FOODS_1_003    11101
3         CA_1      FOODS_1_004    11206
4         CA_1      FOODS_1_005    11101
...        ...              ...      ...
30485     WI_3  HOUSEHOLD_2_512    11101
30486     WI_3  HOUSEHOLD_2_513    11311
30487     WI_3  HOUSEHOLD_2_514    11101
30488     WI_3  HOUSEHOLD_2_515    11352
30489     WI_3  HOUSEHOLD_2_516    11101

[30490 rows x 3 columns]
grid_df after concatenating with item_release_week_df:                                      id        item_id    dept_id   cat_id  \
0         HOBBIES_1_001_CA_1_evaluation  HOBBIES_1_001  HOBBIES_1  HOBBIES   
1         HOBBIES_1_002_CA_1_evaluation  HOBBIES_1_002  HOBBIES_1  HOBBIES   
2         HOBBIES_1_003_CA_1_evaluation  HOBBIES_1_003  HOBBIES_1  HOBBIES   
3         HOBBIES_1_004_CA_1_evaluation  HOBBIES_1_004  HOBBIES_1  HOBBIES   
4         HOBBI

In [19]:
# これで、いくつかの行をカットして安全なメモリにできます
# remove the rows whose release week is earlier than 'wm_yr_wk' when products should not be started  to be sold.
grid_df = grid_df[grid_df['wm_yr_wk'] >= grid_df['release']]
grid_df = grid_df.reset_index(drop=True)

# メモリ使用量を確認しましょう
print("{:>20}: {:>8}".format('Original grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))
print(f"grid_df after removing uneccesary rows: {grid_df}")


    Original grid_df:   1.2GiB
grid_df after removing uneccesary rows:                                      id        item_id    dept_id   cat_id  \
0         HOBBIES_1_008_CA_1_evaluation  HOBBIES_1_008  HOBBIES_1  HOBBIES   
1         HOBBIES_1_009_CA_1_evaluation  HOBBIES_1_009  HOBBIES_1  HOBBIES   
2         HOBBIES_1_010_CA_1_evaluation  HOBBIES_1_010  HOBBIES_1  HOBBIES   
3         HOBBIES_1_012_CA_1_evaluation  HOBBIES_1_012  HOBBIES_1  HOBBIES   
4         HOBBIES_1_015_CA_1_evaluation  HOBBIES_1_015  HOBBIES_1  HOBBIES   
...                                 ...            ...        ...      ...   
47735392    FOODS_3_823_WI_3_evaluation    FOODS_3_823    FOODS_3    FOODS   
47735393    FOODS_3_824_WI_3_evaluation    FOODS_3_824    FOODS_3    FOODS   
47735394    FOODS_3_825_WI_3_evaluation    FOODS_3_825    FOODS_3    FOODS   
47735395    FOODS_3_826_WI_3_evaluation    FOODS_3_826    FOODS_3    FOODS   
47735396    FOODS_3_827_WI_3_evaluation    FOODS_3_827    FOODS_3    FO

In [20]:
# 特徴量の1つとしてリリース週を維持する必要がありますか？ 
# 良いCVだけが答えを出すことができます。 
# リリース値を縮小してみましょう。 
# 最小変換はここでは役に立たない
# int16→integer（-32768から32767） grid_df ['release'].max（）→int16のような変換は。
# しかし、必要な場合に備えて、変換するある方法があります。
grid_df['release'] = grid_df['release'] - grid_df['release'].min()
grid_df['release'] = grid_df['release'].astype(np.int16)

# メモリ使用量をもう一度確認してみましょう
print("{:>20}: {:>8}".format('Reduced grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))
print(f"grid_df:{grid_df}")

     Reduced grid_df:   1.2GiB
grid_df:                                     id        item_id    dept_id   cat_id  \
0         HOBBIES_1_008_CA_1_evaluation  HOBBIES_1_008  HOBBIES_1  HOBBIES   
1         HOBBIES_1_009_CA_1_evaluation  HOBBIES_1_009  HOBBIES_1  HOBBIES   
2         HOBBIES_1_010_CA_1_evaluation  HOBBIES_1_010  HOBBIES_1  HOBBIES   
3         HOBBIES_1_012_CA_1_evaluation  HOBBIES_1_012  HOBBIES_1  HOBBIES   
4         HOBBIES_1_015_CA_1_evaluation  HOBBIES_1_015  HOBBIES_1  HOBBIES   
...                                 ...            ...        ...      ...   
47735392    FOODS_3_823_WI_3_evaluation    FOODS_3_823    FOODS_3    FOODS   
47735393    FOODS_3_824_WI_3_evaluation    FOODS_3_824    FOODS_3    FOODS   
47735394    FOODS_3_825_WI_3_evaluation    FOODS_3_825    FOODS_3    FOODS   
47735395    FOODS_3_826_WI_3_evaluation    FOODS_3_826    FOODS_3    FOODS   
47735396    FOODS_3_827_WI_3_evaluation    FOODS_3_827    FOODS_3    FOODS   

         store_id state_

# export the base grid (grid_part_1)

In [21]:
# save the base grid file as pickle for later usage.
_EXPORT_FILE_NAME = "base_grid_for_darker_magic_evaluation.pkl"
print("data export start")
grid_df.to_pickle(os.path.sep.join([str(parent_dir), _DATA_DIR, _EXPORT_FILE_NAME]))

print('data export finished. Size:', grid_df.shape)

data export start
data export finished. Size: (47735397, 10)


# ----- you can split the notebook here -----

# preprocessing df_sell_prices: add features

In [22]:
display(df_sell_prices.head())

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.578125
1,CA_1,HOBBIES_1_001,11326,9.578125
2,CA_1,HOBBIES_1_001,11327,8.257812
3,CA_1,HOBBIES_1_001,11328,8.257812
4,CA_1,HOBBIES_1_001,11329,8.257812


In [23]:
# 基本的な集計を行う
# the max, min, std, mean price among the same item at the same store
df_sell_prices['price_max'] = df_sell_prices.groupby(['store_id','item_id'])['sell_price'].transform('max')
df_sell_prices['price_min'] = df_sell_prices.groupby(['store_id','item_id'])['sell_price'].transform('min')
df_sell_prices['price_std'] = df_sell_prices.groupby(['store_id','item_id'])['sell_price'].transform('std')
df_sell_prices['price_mean'] = df_sell_prices.groupby(['store_id','item_id'])['sell_price'].transform('mean')

print(f"df_sell_prices['price_max']: {df_sell_prices['price_max']}")
# 価格正規化を行う（min-max scaling）。priceのレンジは大きくないので、Logを行う意味なし。（むしろ、一ドル以下の商品が多いので悪い）
df_sell_prices['price_norm'] = df_sell_prices['sell_price']/df_sell_prices['price_max']
print(f"df_sell_prices['price_norm']: {df_sell_prices['price_norm']}")
print(f"df_sell_prices['price_norm'].shape: {df_sell_prices['price_norm'].shape}")

# # since the price distribution is not very skewed or long-tailed, I decided not to take log scale. see: https://www.kaggle.com/headsortails/back-to-predict-the-future-interactive-m5-eda
# df_sell_prices['price_norm'] = np.log1p(df_sell_prices['sell_price'])
# print(f"log1p df_sell_prices['price_norm']: {df_sell_prices['price_norm']}")
# print(f"log1p df_sell_prices['price_norm'].shape: {df_sell_prices['price_norm'].shape}")

# since "pt.fit(sell_price_2d_array)" caused "ValueError: Input contains infinity or a value too large for dtype('float16').", box-cox transformation was discarded: from scipy import stats also did not make propcer transform.
# # https://note.com/mikiokubo/n/n42417e5d0f6c
# # https://gakushukun1.hatenablog.com/entry/2019/04/29/112424
# pt = PowerTransformer(method="box-cox")
# sell_price_2d_array = df_sell_prices['sell_price'].values.reshape(-1,1)
# # data = df["B"].values
# print(f"sell_price_2d_array: {sell_price_2d_array}")
# pt.fit(sell_price_2d_array)
# df_sell_prices['price_norm'] = pt.transform(sell_price_2d_array)
# df_sell_prices['price_norm'].hist()
# print(f"box-coxed df_sell_prices['price_norm']: {df_sell_prices['price_norm']}")
# print(f"box-coxed df_sell_prices['price_norm'].shape: {df_sell_prices['price_norm'].shape}")

# 一部のアイテムはインフレに依存する可能性があります。いくつかのアイテムは非常に「安定」しています
# count the kinds of sell_price per ['store_id','item_id']. the smaller, the more stable the price is.
df_sell_prices['price_nunique'] = df_sell_prices.groupby(['store_id','item_id'])['sell_price'].transform('nunique')
# count the kinds of item_id per ['store_id','sell_price']. the smaller, the less frequent the price is, whether the price is large or small. todo@kensakuokada: this feature may not be necessary.
df_sell_prices['item_nunique'] = df_sell_prices.groupby(['store_id','sell_price'])['item_id'].transform('nunique')

# I would like some "rolling" aggregations but would like months and years as "window"
calendar_prices = df_calendar[['wm_yr_wk','month','year']]
# the rows are duplicated in each week. reduce rows per week
calendar_prices = calendar_prices.drop_duplicates(subset=['wm_yr_wk'])
# add month and year
df_sell_prices = df_sell_prices.merge(calendar_prices[['wm_yr_wk','month','year']], on=['wm_yr_wk'], how='left')

del calendar_prices
gc.collect()

display(df_sell_prices.head())

df_sell_prices['price_max']: 0          9.578125
1          9.578125
2          9.578125
3          9.578125
4          9.578125
             ...   
6841116    1.000000
6841117    1.000000
6841118    1.000000
6841119    1.000000
6841120    1.000000
Name: price_max, Length: 6841121, dtype: float16
df_sell_prices['price_norm']: 0          1.000000
1          1.000000
2          0.862305
3          0.862305
4          0.862305
             ...   
6841116    1.000000
6841117    1.000000
6841118    1.000000
6841119    1.000000
6841120    1.000000
Name: price_norm, Length: 6841121, dtype: float16
df_sell_prices['price_norm'].shape: (6841121,)


,store_id,item_id,wm_yr_wk,sell_price,price_max,price_min,price_std,price_mean,price_norm,price_nunique,item_nunique,month,year
0,CA_1,HOBBIES_1_001,11325,9.578125,9.578125,8.257812,0.152344,8.28125,1.000000,3.0,3,7,2013
1,CA_1,HOBBIES_1_001,11326,9.578125,9.578125,8.257812,0.152344,8.28125,1.000000,3.0,3,7,2013
2,CA_1,HOBBIES_1_001,11327,8.257812,9.578125,8.257812,0.152344,8.28125,0.862305,3.0,5,7,2013
3,CA_1,HOBBIES_1_001,11328,8.257812,9.578125,8.257812,0.152344,8.28125,0.862305,3.0,5,8,2013
4,CA_1,HOBBIES_1_001,11329,8.257812,9.578125,8.257812,0.152344,8.28125,0.862305,3.0,5,8,2013


In [24]:
# 週ごとにシフト in each (['store_id','item_id']) group. todo@kensakuokada: add more momentum by shifting more, which may be better.
df_sell_prices['price_momentum'] = df_sell_prices['sell_price']/df_sell_prices.groupby(['store_id','item_id'])['sell_price'].transform(lambda x: x.shift(1))
#  each sell price/月平均 in ['store_id','item_id','month'] group
df_sell_prices['price_momentum_m'] = df_sell_prices['sell_price']/df_sell_prices.groupby(['store_id','item_id','month'])['sell_price'].transform('mean')
# each sell price/年平均 in ['store_id','item_id','year'] group
df_sell_prices['price_momentum_y'] = df_sell_prices['sell_price']/df_sell_prices.groupby(['store_id','item_id','year'])['sell_price'].transform('mean')

del df_sell_prices['month'], df_sell_prices['year'] # todo@kensakuokada: try not to remove month and year, which may be better.
gc.collect()

display(df_sell_prices.head())

,store_id,item_id,wm_yr_wk,sell_price,price_max,price_min,price_std,price_mean,price_norm,price_nunique,item_nunique,price_momentum,price_momentum_m,price_momentum_y
0,CA_1,HOBBIES_1_001,11325,9.578125,9.578125,8.257812,0.152344,8.28125,1.000000,3.0,3,NaN,1.126953,1.144531
1,CA_1,HOBBIES_1_001,11326,9.578125,9.578125,8.257812,0.152344,8.28125,1.000000,3.0,3,1.000000,1.126953,1.144531
2,CA_1,HOBBIES_1_001,11327,8.257812,9.578125,8.257812,0.152344,8.28125,0.862305,3.0,5,0.862305,0.971680,0.986816
3,CA_1,HOBBIES_1_001,11328,8.257812,9.578125,8.257812,0.152344,8.28125,0.862305,3.0,5,1.000000,1.000000,0.986816
4,CA_1,HOBBIES_1_001,11329,8.257812,9.578125,8.257812,0.152344,8.28125,0.862305,3.0,5,1.000000,1.000000,0.986816


# preprocessing df_sell_prices: concat df_sell_prices with MAIN_INDEX (connecting with other dataframes)

In [25]:
# base columns
original_columns = list(grid_df)
print(f"original_columns: {original_columns}")

grid_df = grid_df.merge(df_sell_prices, on=['store_id','item_id','wm_yr_wk'], how='left')
print(f"grid_df: {grid_df}")

keep_columns = [col for col in list(grid_df) if col not in original_columns]
print(f"keep_columns: {keep_columns}")

grid_df = grid_df[MAIN_INDEX+keep_columns]
grid_df = reduce_mem_usage(grid_df)
print(f"grid_df: {grid_df}")

# We don't need prices_df anymore
del df_sell_prices
gc.collect()


original_columns: ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd', 'sales', 'release', 'wm_yr_wk']
grid_df:                                      id        item_id    dept_id   cat_id  \
0         HOBBIES_1_008_CA_1_evaluation  HOBBIES_1_008  HOBBIES_1  HOBBIES   
1         HOBBIES_1_009_CA_1_evaluation  HOBBIES_1_009  HOBBIES_1  HOBBIES   
2         HOBBIES_1_010_CA_1_evaluation  HOBBIES_1_010  HOBBIES_1  HOBBIES   
3         HOBBIES_1_012_CA_1_evaluation  HOBBIES_1_012  HOBBIES_1  HOBBIES   
4         HOBBIES_1_015_CA_1_evaluation  HOBBIES_1_015  HOBBIES_1  HOBBIES   
...                                 ...            ...        ...      ...   
47735392    FOODS_3_823_WI_3_evaluation    FOODS_3_823    FOODS_3    FOODS   
47735393    FOODS_3_824_WI_3_evaluation    FOODS_3_824    FOODS_3    FOODS   
47735394    FOODS_3_825_WI_3_evaluation    FOODS_3_825    FOODS_3    FOODS   
47735395    FOODS_3_826_WI_3_evaluation    FOODS_3_826    FOODS_3    FOODS   
47735396    FOO

0

# export grid_df having sales_price features and MAIN_INDEX (grid_part_2)

In [26]:
# 今後のモデルトレーニングのためpickleファイルとして保存
_EXPORT_FILE_NAME = "base_grid_with_sales_price_features_for_darker_magic_evaluation.pkl"
print("data export start")
grid_df.to_pickle(os.path.sep.join([str(parent_dir), _DATA_DIR, _EXPORT_FILE_NAME]))
print('data export finished. Size:', grid_df.shape)


data export start
data export finished. Size: (47735397, 13)


# ----- you can split the notebook here -----

# preprocessing df_calendar

In [27]:
grid_df = grid_df[MAIN_INDEX]

# カレンダーを部分的に結合
icols = ['date',
         'd',
         'event_name_1',
         'event_type_1',
         'event_name_2',
         'event_type_2',
         'snap_CA',
         'snap_TX',
         'snap_WI']

grid_df = grid_df.merge(df_calendar[icols], on=['d'], how='left')

# データを縮小する 
# 'snap_'列はboolまたはint8に変換できる 
icols = ['event_name_1',
         'event_type_1',
         'event_name_2',
         'event_type_2',
         'snap_CA',
         'snap_TX',
         'snap_WI']
for col in icols:
    grid_df[col] = grid_df[col].astype('category')

# 日時に変換
grid_df['date'] = pd.to_datetime(grid_df['date'])

# 日付からいくつかの特徴量を作る
grid_df['tm_d'] = grid_df['date'].dt.day.astype(np.int8)
grid_df['tm_w'] = grid_df['date'].dt.week.astype(np.int8)
grid_df['tm_m'] = grid_df['date'].dt.month.astype(np.int8)
grid_df['tm_y'] = grid_df['date'].dt.year
grid_df['tm_y'] = (grid_df['tm_y'] - grid_df['tm_y'].min()).astype(np.int8)
grid_df['tm_wm'] = grid_df['tm_d'].apply(lambda x: math.ceil(x/7)).astype(np.int8)

grid_df['tm_dw'] = grid_df['date'].dt.dayofweek.astype(np.int8)
grid_df['tm_w_end'] = (grid_df['tm_dw']>=5).astype(np.int8)

print(f"grid_df: {grid_df}")

grid_df:                                      id       d       date  event_name_1  \
0         HOBBIES_1_008_CA_1_evaluation     d_1 2011-01-29           NaN   
1         HOBBIES_1_009_CA_1_evaluation     d_1 2011-01-29           NaN   
2         HOBBIES_1_010_CA_1_evaluation     d_1 2011-01-29           NaN   
3         HOBBIES_1_012_CA_1_evaluation     d_1 2011-01-29           NaN   
4         HOBBIES_1_015_CA_1_evaluation     d_1 2011-01-29           NaN   
...                                 ...     ...        ...           ...   
47735392    FOODS_3_823_WI_3_evaluation  d_1969 2016-06-19  NBAFinalsEnd   
47735393    FOODS_3_824_WI_3_evaluation  d_1969 2016-06-19  NBAFinalsEnd   
47735394    FOODS_3_825_WI_3_evaluation  d_1969 2016-06-19  NBAFinalsEnd   
47735395    FOODS_3_826_WI_3_evaluation  d_1969 2016-06-19  NBAFinalsEnd   
47735396    FOODS_3_827_WI_3_evaluation  d_1969 2016-06-19  NBAFinalsEnd   

         event_type_1  event_name_2 event_type_2 snap_CA snap_TX snap_WI  \
0 

In [28]:
# 日付の削除
del grid_df['date']

In [29]:
grid_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47735397 entries, 0 to 47735396
Data columns (total 16 columns):
 #   Column        Dtype   
---  ------        -----   
 0   id            category
 1   d             object  
 2   event_name_1  category
 3   event_type_1  category
 4   event_name_2  category
 5   event_type_2  category
 6   snap_CA       category
 7   snap_TX       category
 8   snap_WI       category
 9   tm_d          int8    
 10  tm_w          int8    
 11  tm_m          int8    
 12  tm_y          int8    
 13  tm_wm         int8    
 14  tm_dw         int8    
 15  tm_w_end      int8    
dtypes: category(8), int8(7), object(1)
memory usage: 1.4+ GB


# export grid_df having sales_price features and MAIN_INDEX (grid_part_3)

In [30]:
_EXPORT_FILE_NAME = "base_grid_with_calendar_features_for_darker_magic_evaluation.pkl"
print("data export start")
grid_df.to_pickle(os.path.sep.join([str(parent_dir), _DATA_DIR, _EXPORT_FILE_NAME]))
print('data export finished. Size:', grid_df.shape)

data export start
data export finished. Size: (47735397, 16)
